In [32]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import torch
from ultralytics import YOLO  # YOLOv5/YOLOv8通用接口
import cv2
import matplotlib.pyplot as plt

In [33]:
# 数据集
DATASET_ROOT = "E:\python\code\machine_vision\experiment\experiment4\shared_bicycle"
# 数据集配置文件
YAML_PATH = os.path.join(DATASET_ROOT, "cycledata.yaml")
# YOLOv5模型权重
MODEL_WEIGHT = "yolov5s.pt"
#训练好的模型
model_save_path = os.path.join("bicycle_experiment", "shared_bicycle", "weights")
# 训练参数
EPOCHS = 50     # 训练轮数
BATCH_SIZE = 4  # 批次大小
IMG_SIZE = 640  # 输入图片尺寸
DEVICE = 0 if torch.cuda.is_available() else "cpu"

In [34]:
if not os.path.exists(YAML_PATH):
        print(f"错误：找不到cyckedata.yaml → {YAML_PATH}")
        print("请核对DATASET_ROOT路径是否正确！")

In [35]:
# 训练YOLOv5模型
def train_yolov5():
    # 加载预训练的YOLOv5模型
    model = YOLO(MODEL_WEIGHT)
    # 开始训练
    results = model.train(
        data=YAML_PATH,
        epochs=EPOCHS,
        batch=BATCH_SIZE,
        imgsz=IMG_SIZE,
        device=DEVICE,
        project="bicycle_experiment",  # 训练结果保存目录
        name="shared_bicycle",  # 实验名称（区分不同训练）
        save=True,  # 保存最优模型
        val=True  # 训练时同步验证
    )
    # 输出训练结果
    
    print(f"\n训练完成！模型权重保存在：{model_save_path}")
    print(f"最优权重：{os.path.join(model_save_path, 'best.pt')}")
    print(f"最后一轮权重：{os.path.join(model_save_path, 'last.pt')}")
    return model

In [36]:
# 验证模型精度
def validate_model(model):
    results = model.val(
        data=YAML_PATH,
        imgsz=IMG_SIZE,
        device=DEVICE,
        split="val"
    )
    # 打印核心精度指标
    print("\n===== 验证集精度结果 =====")
    print(f"整体mAP@0.5: {results.box.map50:.4f}")
    print(f"共享单车类别精度: {results.box.ap50[0]:.4f}")
    return results

In [37]:
# 训练模型
trained_model = train_yolov5()
# 验证模型精度
validate_model(trained_model)

PRO TIP  Replace 'model=yolov5s.pt' with new 'model=yolov5su.pt'.
YOLOv5 'u' models are trained with https://github.com/ultralytics/ultralytics and feature improved performance vs standard YOLOv5 models trained with https://github.com/ultralytics/yolov5.
New https://pypi.org/project/ultralytics/8.4.7 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.241  Python-3.10.13 torch-2.6.0+cu118 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=False, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=E:\python\code\machine_vision\experiment\experiment4\shared_bicycle\cycledata.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscr

RuntimeError: This is not allowed since there's already a kernel registered from python overriding inductor_seed's behavior for BackendSelect dispatch key and prims namespace.

In [29]:
# 测试模型效果
def predict_image(model, img_path):
    if not os.path.exists(img_path):
        print(f"错误：图片路径不存在 → {img_path}")
        return
    # 执行预测
    results = model(img_path)
    # 绘制边界框、类别、置信度
    res_plotted = results[0].plot()
    # 打印预测详情
    print("\n===== 预测结果 =====")
    for box in results[0].boxes:
        cls_name = model.names[int(box.cls[0])]  # 获取类别名
        conf = float(box.conf[0])  # 获取置信度
        print(f"检测到：{cls_name}，置信度：{conf:.4f}")
    res_rgb = cv2.cvtColor(res_plotted, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(12, 8))  # 设置画布大小
    plt.imshow(res_rgb)
    plt.axis('off')  # 关闭坐标轴，只显示图片
    plt.title('YOLO Prediction Result', fontsize=16)  # 添加标题
    plt.tight_layout()  # 调整布局
    plt.show()  # 显示图片

In [30]:
#加载已保存的训练模型
def load_trained_model(model_path):
    if not os.path.exists(model_path):
        raise FileNotFoundError(f"找不到已保存的模型文件 → {model_path}")
    print(f"\n正在加载已保存的模型：{model_path}")
    model_path = os.path.join(model_path, "best.pt")
    model = YOLO(model_path)  # 加载本地权重文件
    return model

In [31]:
# 测试单张图片
test_img_path = os.path.join(DATASET_ROOT, "bike_test.jpg")
trained_model = load_trained_model(model_save_path)
predict_image(trained_model, test_img_path)


正在加载已保存的模型：bicycle_experiment\shared_bicycle\weights
WARNING GitHub assets check failure for https://api.github.com/repos/ultralytics/assets/releases/tags/v8.3.0: 403 rate limit exceeded
WARNING GitHub assets check failure for https://api.github.com/repos/ultralytics/assets/releases/latest: 403 rate limit exceeded


FileNotFoundError: [Errno 2] No such file or directory: 'bicycle_experiment\\shared_bicycle\\weights\\best.pt'